#### Construcción del knowledge graph a partir de pdd data

* **PDD Graph: Bidging Electronic Medical Records and Biomedical Knowledge Graphs via Entity Linking**
    * http://kmap.xjtudlc.com/pdd
    * https://github.com/wangmengsd/pdd-graph
    * Colección de triplas con uris.
    * Vincula con ICD9, DrugBank e, indirectamente, con UMLS.
    * Viejo.
    
* **Building a knowledge graph to enable precision medicine**
    * https://www.nature.com/articles/s41597-023-01960-3#code-availability
    * https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/IXA7BM
    * https://github.com/mims-harvard/PrimeKG
    * Parece bastante completo, aunque no se si tiene la parte de UMLS que quería agregar.
    * Pareciera que no tiene info del ICD. Raro dado que tiene información de enfermedades.
    * Lo que definitivamente NO tiene es lo de MIMIC, pero si están los ids de las ontologías, se puede agregar sin mucho problema.
    * Actualizado a 2023.
    
* **Integrating and formatting biomedical data as pre-calculated knowledge graph embeddings in the Bioteque**
    * https://bioteque.irbbarcelona.org/downloads
    * https://gitlabsbnb.irbbarcelona.org/bioteque/bioteque
    * Tiene scripts.
    * Tiene notebooks de ejemplo.
    * El KG no está (solo los nodos), hay que reconstruirlo.
    * Parece ser a más bajo nivel (gen y eso).
  
Probablemente haya que mergear ambos KG, y completar lo que pueda faltar.

#### Reconstruyendo: "Building a knowledge graph to enable precision medicine"

In [ ]:
dir_path = "./"

In [ ]:
import pandas as pd
import networkx as nx
from tqdm.notebook import tqdm

In [ ]:
g = nx.DiGraph()

In [ ]:
# nodes

df_nodes = pd.read_csv(dir_path + 'nodes.csv')

id_type_to_index = {}

for i in tqdm(range(0,len(df_nodes))):
    g.add_node(df_nodes['node_index'].values[i],node_id = df_nodes['node_id'].values[i].split('_'), # cada nodo puede englobar varios id
                                                node_type = df_nodes['node_type'].values[i],
                                                node_name = df_nodes['node_name'].values[i],
                                                node_source = df_nodes['node_source'].values[i],
                                            )
del df_nodes
g.number_of_nodes()

In [ ]:
# edges -- del edges.csv, kg.csv o kg_giant.csv
# kg_giant tiene más relaciones, pero los nodos no están en el grafo (solo da los ids, hay ids repetidos y hay index que agrupan más de un id)
# kg tiene los index, con lo que la búsqueda es directa

df_kg = pd.read_csv(dir_path + 'kg.csv')

for i in tqdm(range(0,len(df_kg))):    
    x = df_kg['x_index'].values[i]
    y = df_kg['y_index'].values[i]

    g.add_edge(x,y,relation=df_kg['relation'].values[i], display_relation=df_kg['display_relation'].values[i]) # las otras columnas son info de los nodos

g.number_of_edges()

In [ ]:
# drug_features 

df_drugs = pd.read_csv(dir_path + 'drug_features.csv')
df_drugs = df_drugs.fillna('')
for i in tqdm(range(0,len(df_drugs))):
    for j in range(1,len(df_drugs.columns)):
        c = df_drugs.columns[j]
        if len(df_drugs[c].values[i]) > 0: # si tiene algo, lo agrego
            if c == 'pathway':
                g.nodes[df_drugs['node_index'].values[i]][c] = [x.strip() for x in df_drugs[c].values[i].split(';')]
            else:
                g.nodes[df_drugs['node_index'].values[i]][c] = df_drugs[c].values[i]
    
del df_drugs

In [ ]:
# disease_features 
# -- los ids pueden aparecer más de una vez con distinta información!!!
# -- los ids que están agrupados aparecen con distintos nombres !!
# -- hay que crear las relaciones separadas para cada posible id del nodo de nombre a nombre

# df_disease = pd.read_csv(dir_path + 'disease_features.csv')
# df_disease = df_disease.fillna('')

for i in tqdm(range(0,len(df_disease))): # nodo : {nodo_id : {attributes}}
    dicti = {}
    nid = df_disease['mondo_id'].values[i]
    nindex = df_disease['node_index'].values[i] 
    for c in df_disease.columns[3:]:
        
        vv = df_disease[c].values[i]
        if len(vv) == 0:
            continue
        
        if nid not in g.nodes[nindex]: # si no existe este node_id
            g.nodes[nindex][nid] = {}
            if c != 'group_id_bert':
                g.nodes[nindex][nid][c] = set([vv])
            else:
                g.nodes[nindex][nid][c] = set(vv.split('_'))
        else: # si ya lo tenía
            if c in g.nodes[nindex][nid]:
                if c != 'group_id_bert':
                    g.nodes[nindex][nid][c].add(vv)
                else:
                    g.nodes[nindex][nid][c].update(vv.split('_')) 
            else:
                if c != 'group_id_bert':
                    g.nodes[nindex][nid][c] = set([vv])
                else:
                    g.nodes[nindex][nid][c] = set(vv.split('_'))
    
del df_disease 

In [ ]:
# fix in case we forgot to do it in the previous step
for n in tqdm(g.nodes(data=True)):
    if n[1]['node_type'] == 'disease':
        for k,v in n[1].items():
            if not isinstance(v,dict):
                continue
            tr = set()
            for kk,vv in v.items():
                if isinstance(vv,list) or isinstance(vv,set):
                    g.nodes[n[0]][k][kk] = set([x for x in vv if len(x) > 0])
                if len(g.nodes[n[0]][k][kk]) == 0:
                    tr.add(kk) 
            for kk in tr:
                del g.nodes[n[0]][k][kk]

In [ ]:
# store full graph as pickle
nx.write_gpickle(g,dir_path + 'primeKG_original.gpickle')

In [ ]:
# construcción and save triplas
# -- itera por todos los arcos -- agrega la relación entre el origen y el destino, pero por nombre
# -- agrega origen con cada uno de los destinos (por nombre)
# -- agrega relación is_a drug/disease, ... (nodos)
# -- agrega relación same_group (para vincular los ids que están en el mismo grupo) (nodos)
# -- disease, agregar symptoms
# para las drugs se conoce su id en DrugBank, para lo diseases no se conoce nada

from collections import deque

triples = deque()
for e in tqdm(g.edges(data=True)):
    triples.append((g.nodes[e[0]]['node_name'],e[2]['relation'],g.nodes[e[1]]['node_name']))

triples

In [ ]:
for n in g.nodes(data=True):
    
    triples.append((n[1]['node_name'],'is_a',n[1]['node_type']))
    
#     if n[1]['node_type'] == 'disease':
#         for k,v in n[1].items():
#             if not isinstance(v,dict):
#                 continue
#             if 'mayo_symptoms' in v:
#                 print(k,v['mayo_symptoms'])
#                 break

In [ ]:
import pickle

with open('primeKG_triples.pickle','wb') as file:
    pickle.dump(triples,file)

#### Reconstruyendo: "PDD Graph: Bidging Electronic Medical Records and Biomedical Knowledge Graphs via Entity Linking"

In [ ]:
import pandas as pd
import networkx as nx
from tqdm.notebook import tqdm
from rdflib.graph import Graph

In [ ]:
path_dir = './pdd_nt/'

In [ ]:
g = nx.DiGraph()

In [ ]:
# age_gender.nt
# -- genera los nodos de resource
# -- les agrega dos propiedades, type, gender y age (por ahora no crean relaciones, solo atributos)

gnt = Graph()
gnt.parse(path_dir + "age_gender.nt", format="nt")

for e in tqdm(gnt):
    
    node = e[0].split('/')[-1]
    if not g.has_node(node):
        g.add_node(node)
    
    att = e[1].split('/')[-1]
    g.nodes[node][att] = e[2][0] if att != 'age' else int(e[2][0])
    
del gnt

In [ ]:
# BMI_information.nt
# idem que el anterior, son tres propiedades float. NO todos las tienen

gnt = Graph()
gnt.parse(path_dir + "BMI_information.nt", format="nt")

for e in tqdm(gnt):
    
    node = e[0].split('/')[-1]
    if not g.has_node(node):
        g.add_node(node)
    
    g.nodes[node][e[1].split('/')[-1]] = float(e[2][0])

del gnt

In [ ]:
# patients_basic.nt
# agrega nodos de paciente y algunas de sus propiedades
# agrega relación entre hospital admision y patient

gnt = Graph()
gnt.parse(path_dir + "patients_basic.nt", format="nt")

for e in tqdm(gnt):
    source = e[0].split('/')[-1]
    rel = e[1]
    dest = e[2]
    if rel.endswith('#type'):
        if not g.has_node(source):
            g.add_node(source)
        g.nodes[source]['type_'] = dest.split('/')[-1] # TODO!
    elif rel.endswith('property:hospital_admission_id') or rel.endswith('property:patient_id'):
        pass
    else:
        dest = dest.split('/')[-1]
        if source == dest:
            continue
        if not g.has_node(source):
            g.add_node(source,type_='vocabulary:Admission')
            # add hospital visit
        if not g.has_node(dest):
            g.add_node(dest,type_='vocabulary:Patient')
        g.add_edge(source,dest,type_='hospital_admission_id')
#         g.add_edge(dest,source,type_='hospital_admission_id')

del gnt
# if e[1].endswith('#type'): se chequea existencia de nodo y se agrega el atributo al nodo correspondiente
# if e[1].endswith('hospital_admission_id'): se agrega relación entre paciente --> hospital id

In [ ]:
nx.write_gpickle(g, path_dir + 'PDD_age_bmi_patients.gpickle') # si rompo a partir de acá, puedo levantar este

In [ ]:
# drug_patients.nt
# -- define relación entre hospital id --> drug name, hospital id --> drug id
# -- solo considerar el drug id, el drug name lo podríamos agregar luego, no debería ser en este punto una relación, sino un atributo
# puedo tener una lista donde acumulo nombres y siempre saco el primero ? asumiendo que siempre están en el mismo orden

gnt = Graph()
gnt.parse(path_dir + "drug_patients.nt", format="nt")

for e in tqdm(gnt):
    if e[1].endswith('take_drug_name'): 
        continue
    drug = e[2].split(':')[-1]
    hc = e[0].split('/')[-1]
    if not g.has_node(drug):
        g.add_node(drug,type_='drug')
    g.add_edge(hc,drug,type_='take_drug_id')
    
del gnt

In [ ]:
nx.write_gpickle(g, path_dir + 'PDD_age_bmi_patients_drug.gpickle') # si rompo a partir de acá, puedo levantar este

In [ ]:
# ddi.nt
# -- son relaciones entre drugs
# -- se agregan los nodos con el type: drug (los ids de nodo son lo mismo que hay que buscar en el drugbank)

gnt = Graph()
gnt.parse(path_dir + "ddi.nt", format="nt")

for e in tqdm(gnt):
    
    drug1 = e[0].split(':')[-1]
    drug2 = e[2].split(':')[-1]
    
    if not g.has_node(drug1):
        g.add_node(drug1,type_='drug')
    
    if not g.has_node(drug2):
        g.add_node(drug2,type_='drug')
    
    g.add_edge(drug1,drug2,type_='interact')
    
del gnt

In [ ]:
nx.write_gpickle(g, path_dir + 'PDD_age_bmi_patients_drug_interact.gpickle') # si rompo a partir de acá, puedo levantar este

In [ ]:
# diagnose_icd_information.nt
# -- define como nodos los códigos de ICD-9
# -- define la relación entre los diagnósticos y la hospital id --> código ICD9

# gnt = Graph()
# gnt.parse(path_dir + "diagnose_icd_information.nt", format="nt")

i = 0
for e in tqdm(gnt):
    
    hc = e[0].split('/')[-1]
    diagnose = '/'.join(e[2].split('/')[-2:])
    
    if not g.has_node(diagnose):
        g.add_node(diagnose,type_='ICD_diagnose')
        
    g.add_edge(hc,diagnose,type_='diagnoses_icd9')
    
del gnt

In [ ]:
nx.write_gpickle(g, path_dir + 'PDD_age_bmi_patients_drug_interact_diagnoses.gpickle') # si rompo a partir de acá, puedo levantar este

In [ ]:
for e in g.edges(data=True):
    if 'diagnoses_icd9' in e[2]['type_']:
        print(e)
        break

In [ ]:
# prescriptions.nt
# -- agrega nodos de prescription
# -- agrega relación entre hospital id --> prescription
# -- agrega atributos al prescription

# Big file (40956557 rows). Loading everything at once does not seem to be a good idea :)

gnt = Graph()

with open(path_dir + "prescriptions.nt",'r') as file:
    
    for line in tqdm(file):
        
        if 'start_date' in line or 'end_date' in line:
            continue
        
        gnt.parse(data=line, format='nt')
        
        if len(gnt) == 5000: # process every 10000
            
            for e in gnt:
                if e[1].endswith('#type'): # agregamos la prescription al graph con el vocabulary
                    pres = e[0].split('/')[-1]
                    if not g.has_node(pres):
                        g.add_node(pres,type_=e[2].split('/')[-1])
                elif e[1].endswith('has_prescription'): # agregamos el edge entre prescription y hc. hc tiene que estar, el otro se puede agregar luego
                    hc = e[0].split('/')[-1]
                    pres = e[2].split('/')[-1]
                    if not g.has_node(hc): # acá puede que falte el nodo y por eso lo agrega sin tipo!!
                        continue
                    if not g.has_node(pres):
                         g.add_node(pres,type_='vocabulary:Prescription')
                            
                    g.add_edge(hc,pres,type_='has_prescription')
                    
                elif e[1].endswith('start_date') or e[1].endswith('end_date'):
                    continue
                    
                elif e[1].endswith('duration_days'): # add property to prescription
                    pres = e[0].split('/')[-1]
                    if not g.has_node(pres):
                        g.add_node(pres,type_='vocabulary:Prescription') 
                    g.nodes[pres]['duration_days'] = e[2]
                    
                elif e[1].endswith('drug_type'): # add property to prescription
                    pres = e[0].split('/')[-1]
                    if not g.has_node(pres):
                        g.add_node(pres,type_='vocabulary:Prescription') 
                    g.nodes[pres]['drug_type'] = e[2]
                    
                elif e[1].endswith('take_drugbank_id'): # add relaion from prescription to drug
                    pres = e[0].split('/')[-1]
                    drug = e[2].split('/')[-1]
                    if not g.has_node(pres):
                        g.add_node(pres,type_='vocabulary:Prescription') 
                    if not g.has_node(drug):
                        g.add_node(drug,type_='drug') 
                    g.add_edge(pres,drug,type_='take_drugbank_id')        
                    
                elif e[1].endswith('dose'): # add property to prescription
                    pres = e[0].split('/')[-1]
                    if not g.has_node(pres):
                        g.add_node(pres,type_='vocabulary:Prescription') 
                    g.nodes[pres]['dose'] = e[2]
                
            del gnt
            gnt = Graph()
del gnt

In [ ]:
import pickle
import gc

i = 0
dicti_nodes = {}
for n in tqdm(g.nodes(data=True)):
    dicti_nodes[n[0]] = n[1]
    i += 1
    if i % 1000000 == 0:
        with open(path_dir + '__PDD_nodes_'+str(i)+'.pickle','wb') as file:
            pickle.dump(dicti_nodes,file)
        dicti_nodes.clear()
        gc.collect()
        
if len(dicti_nodes) > 0:
    with open(path_dir + '__PDD_nodes_'+str(i)+'.pickle','wb') as file:
        pickle.dump(dicti_nodes,file)
    dicti_nodes.clear()

In [ ]:
import pickle
import gc

i = 0
dicti_nodes = {}
for e in tqdm(g.edges(data=True)):   
    dicti_nodes[e[0]+'___'+e[1]] = e[2]
    i += 1
    if i % 500000 == 0:
        with open(path_dir + '__PDD_edges_'+str(i)+'.pickle','wb') as file:
            pickle.dump(dicti_nodes,file)
        dicti_nodes.clear()
        gc.collect()

if len(dicti_nodes) > 0:
    with open(path_dir + '__PDD_edges_'+str(i)+'.pickle','wb') as file:
        pickle.dump(dicti_nodes,file)
    dicti_nodes.clear()
    gc.collect()
# dicti_nodes

In [ ]:
# las keys se pueden modificar de una, esto no genera ningún inconveniente
import os
keys_to_modify = set()
for ff in tqdm(os.listdir(path_dir)):
    if 'edges' not in ff:
        continue
    dd = pd.read_pickle(path_dir + ff)
    keys_to_modify.update([x for x in dd if 'drugbank:' in x])
    
len(keys_to_modify)

In [ ]:
# no hay superposición, se pueden cambiar sin problema
edges = {}
for ff in tqdm(os.listdir(path_dir)):
    if 'edges' not in ff:
        continue
    dd = pd.read_pickle(path_dir + ff)
    edges.update(dd)
len(edges)

In [ ]:
edges_keys = set(edges.keys())
for e in tqdm(edges_keys):
    if 'drugbank:' in e:
        f = e.replace('drugbank:','')
        edges[f] = dict(edges[e])
        del edges[e]
len(edges)

In [ ]:
# chequear que el viejo ya no está y que ahora está el nuevo
for k in tqdm(keys_to_modify):
#     print(k)
    kk = k.replace('drugbank:','')
#     print(k in edges, kk in edges)
    if kk not in edges:
        edges[f] = dict(edges[e])
    if k in edges:
        del edges[k]
#     print(k in edges, kk in edges)
#     print(edges[kk])
len(edges)

In [ ]:
import pickle
with open(path_dir + '__PDD_edges.pickle','wb') as file:
    pickle.dump(edges,file)

In [ ]:
# hay 2473 drogas
# esto está un toque más complejo... no se si ambos van a estar en el mismo archivo, ni si van a tener la misma información
# tendría que unir los dos dicts
# hay drugs en todos los files !! chequear que están los dos

import os

drug_nodes = set()
for ff in tqdm(os.listdir(path_dir)):
    if 'nodes' not in ff:
        continue
    dd = pd.read_pickle(path_dir + ff)
    aa = [x for x in dd if x.startswith('D') or x.startswith('d')]
    del dd
    print(len(aa))
    drug_nodes.update(aa)
len(drug_nodes)

In [ ]:
for ff in tqdm(os.listdir(path_dir)):
    if 'nodes' not in ff:
        continue
    dd = pd.read_pickle(path_dir + ff)
    nodes = set(dd.keys())
    for n in nodes:
        if 'drug' in n:
            nn = n.replace('drugbank:','')
            if nn in missing_nodes:
                dd[nn] = dict(dd[n]) # único caso en el que hay que agregarlo
            del dd[n]
    with open(path_dir + ff,'wb') as file:
        pickle.dump(dd,file)

In [ ]:
for d in drug_nodes:
    if d.startswith('drugbank:'):
        if d.split(':')[-1] not in drug_nodes:
            print(d)

In [ ]:
missing_nodes = set()
for k in tqdm(keys_to_modify):
    ll = k.split('___')[-1].replace('drugbank:','')
    if ll not in drug_nodes:
        missing_nodes.add(ll)
missing_nodes

In [ ]:
# todos los que están mal son prescriptions

path_nodes = path_dir + 'pdd_nt/'

for ff in tqdm(os.listdir(path_nodes)):
    
    if '__PDD_nodes' not in ff:
        continue
    print(ff)
    nodes = pd.read_pickle(path_nodes + ff)   
    for n,data in tqdm(nodes.items()):
        if 'type_' not in data:
            if not n.startswith('pres'):
                continue
            changed = True
            for k in data:
                data[k] = str(data[k])
            data['type_'] = 'vocabulary:Prescription'
            nodes[n] = dict(data)

    if changed:
        print('Updating...')
        with open(path_nodes + ff,'wb') as file:
            pickle.dump(nodes,file)

In [ ]:
# save del graph
# nx.write_gpickle(g, path_dir + 'PDD_original_graph.gpickle') # si rompo a partir de acá, puedo levantar este

In [ ]:
# construcción de las triplas
# se guardan los edges del grafo
# se guardan algunos de los atributos de los nodos del grafo
# no hace falta que tengan _ los nombres, pueden ser más de una palabra
# hay atributos para hospital id y prescription 

In [ ]:
for n in g.nodes(data=True):
    print(n)
    

In [ ]:
# ICD9CM.ttl
# -- tiene 22533 diagnósticos
# -- Arrancar por levantar el xml/ontology
# si la class es de nuestro interés, le sacamos los valores
# agregamos a las clases de interés las de subClass
# al final, si nos quedan cosas por procesar, hacemos las queries a la web

In [ ]:
import urllib.request, urllib.error, urllib.parse
import json
import os

REST_URL = "http://data.bioontology.org"
API_KEY = "160b4fe8-01d2-4b79-8c76-e930336fec68"

def get_json(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    return json.loads(opener.open(url).read())

In [ ]:
missing = set()
diagnosis = {}

for n in g.nodes(data=True):
    if n[1]['type'] == 'ICD_diagnose' and not n[0] in diagnosis:
        missing.add(n[0])
        
while len(missing) != 0:
    icd = missing.pop() # saca elemento random
    # search for item
    json = get_json(REST_URL + "/search?q=" + term + "&include=properties")["collection"]
    # obtenemos las properties
    diagnosis[icd] = {}
    if True: # if es subclass de alguno, agregamos esa clase a la búsqueda 
        missing.add(None)

In [ ]:
# DrugBank -- es un XML
# -- Supuestamente pude bajar toda la base de datos
# -- No se en qué formato estará la base

In [ ]:
# UMLS para buscar los CUI
# -- Está el thesaurus completo para bajar (3.9GB)
# -- No se formato

umls_api_key = ''

In [ ]:
# https://uts-ws.nlm.nih.gov/rest/content/current/CUI/C0042904/atoms?apiKey=a6f141b2-6c07-4d21-868e-6d316346dfbd
# -- De acá buscamos los atoms que tengan language == 'ENG'
# -- De esos, buscamos el sourceConcept y buscamos sobre esa url
# -- Esa url va a tener otra lista más de urls a procesar
# -- Sigo sin entender cuáles son las relaciones importantes para sacar de acá y cómo llegar a ellas